In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from collections import Counter
import nflgame
from src.data.make_raw_data_3 import main

In [5]:
main()

Last updated: 2018-08-11 18:28:22.074025
Updating (2017, PRE, 0)...
Updating (2017, PRE, 1)...
Updating (2017, PRE, 2)...
Updating (2017, PRE, 3)...
Updating (2017, PRE, 4)...
Updating (2017, REG, 1)...
Updating (2017, REG, 2)...
Updating (2017, REG, 3)...
Updating (2017, REG, 4)...
Updating (2017, REG, 5)...
Updating (2017, REG, 6)...
Updating (2017, REG, 7)...
Updating (2017, REG, 8)...
Updating (2017, REG, 9)...
Updating (2017, REG, 10)...
Updating (2017, REG, 11)...
Updating (2017, REG, 12)...
Updating (2017, REG, 13)...
Updating (2017, REG, 14)...
Updating (2017, REG, 15)...
Updating (2017, REG, 16)...
Updating (2017, REG, 17)...
Updating (2017, POST, 1)...
Updating (2017, POST, 2)...
Updating (2017, POST, 3)...
Updating (2017, POST, 4)...

> /Users/benlindsay/MEGAsync/data-projects/nfl-dash/src/data/make_raw_data_3.py(127)main()
-> if stat in SCORING:
(Pdb) stat
u'defense_sk'
(Pdb) p
*** SyntaxError: SyntaxError('unexpected EOF while parsing', ('<string>', 0, 0, ''))
(Pdb) print 

BdbQuit: 

In [13]:
d = Counter()
d

Counter()

In [15]:
d['NE'] += 1
d['WA'] += 2.5
d

Counter({'NE': 2, 'WA': 2.5})

In [19]:
def defense_fantasy_points(game, home=True):
    if home:
        team = game.home
        opposing_points = game.score_away
    else:
        team = game.away
        opposing_points = game.score_home
    if opposing_points == 0:
        fantasy_points = 10
    elif opposing_points < 7:
        fantasy_points = 7
    elif opposing_points < 14:
        fantasy_points = 4
    elif opposing_points < 21:
        fantasy_points = 1
    elif opposing_points < 28:
        fantasy_points = 0
    elif opposing_points < 35:
        fantasy_points = -1
    else:
        fantasy_points = -4
    return fantasy_points

In [22]:
defense_scores = Counter()
game = nflgame.games(2017, week=1)[0]
print game.score_away, game.score_home
print game.home, defense_fantasy_points(game, home=True)
print game.away, defense_fantasy_points(game, home=False)

42 27
NE -4
KC 0


In [28]:
for game in nflgame.games(2017, week=1):
    for p in game.drives.plays():
        print p
        print dir(p)
        print p.touchdown
        print p.kicking_yds
        break
    break

(KC, KC 35, Q1) C.Santos kicks 64 yards from KC 35 to NE 1. D.Lewis to NE 27 for 26 yards (K.Pierre-Louis, D.Harris).
['_Play__players', '__class__', '__delattr__', '__dict__', '__doc__', '__eq__', '__format__', '__getattr__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_stats', 'data', 'defense_ast', 'defense_tkl', 'defense_tkl_primary', 'desc', 'down', 'drive', 'events', 'has_player', 'home', 'kicking_tot', 'kicking_yds', 'kickret_ret', 'kickret_yds', 'note', 'players', 'playid', 'team', 'time', 'touchdown', 'yardline', 'yards_togo']
False
64


In [33]:
for game in nflgame.games(2017, week=1):
    # print dir(game)
    print

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattr__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', 'away', 'data', 'down', 'eid', 'game_over', 'gamekey', 'home', 'is_home', 'loser', 'max_player_stats', 'nice_score', 'playing', 'rawData', 'save', 'schedule', 'score_away', 'score_away_q1', 'score_away_q2', 'score_away_q3', 'score_away_q4', 'score_away_q5', 'score_home', 'score_home_q1', 'score_home_q2', 'score_home_q3', 'score_home_q4', 'score_home_q5', 'scores', 'season', 'stats_away', 'stats_home', 'time', 'togo', 'winner']
['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattr__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', 'away', 'da

In [9]:
for game in nflgame.games(2017, week=1):
    for p in game.drives.plays().filter(touchdown=True).players().touchdowns().defense():
        print(p)

A.Chickillo
J.Bethel
M.Killebrew
D.Fowler
R.Kerrigan
F.Cox
T.Johnson
L.Joyner


In [25]:
import nflgame

scoring = {
    # Passing
    'passing_yds' : lambda x : x*.04,
    'passing_tds' : lambda x : x*4,
    'passing_twoptm'  : lambda x : x*2,
    # Rushing
    'rushing_yds' : lambda x : x*.1,
    'rushing_tds' : lambda x : x*6,
    'kickret_tds' : lambda x : x*6,
    'rushing_twoptm' : lambda x : x*2,
    # Receiving
    'receiving_tds' : lambda x : x*6,
    'receiving_yds' : lambda x : x*.1,
    'receiving_rec' : lambda x : x*1,
    'receiving_twoptm' : lambda x : x*2,
    # Kicker
    'kicking_fgm_yds' : lambda x : (5 if x >= 50 else (4 if x >= 40 else 3)),
    'kicking_xpmade' : lambda x : x*1,
    'kicking_xpmissed' : lambda x : x*-1,
    'kicking_fgmissed' : lambda x : x*-1,
    # Various
    'fumbles_lost' : lambda x : x*-2, 
    'passing_ints' : lambda x : x*-2,
}

def score_player(player):
    score = 0
    for stat in player._stats:
        if stat in scoring:
            score += scoring[stat](getattr(player,stat))    
    return score

players = nflgame.combine_game_stats(nflgame.games(2017, 1))
for p in players:
    score = score_player(p)
    print p.name,p.team,p.guess_position,score

T.Brady NE QB 10.68
D.Lewis NE RB 0.9
R.Burkhead NE RB 3.3
C.Hogan NE WR 3.5
J.White NE RB 9.8
M.Gillislee NE RB 22.5
B.Cooks NE WR 11.8
R.Gronkowski NE TE 5.3
D.Amendola NE WR 16.0
D.Andrews NE C 0
D.McCourty NE FS 0
S.Gostkowski NE K 3
R.Allen NE P 0
D.Harris NE ILB 0
L.Guy NE DE 0
T.Flowers NE DE 0
D.Hightower NE OLB 0
A.Branch NE DT 0
C.Marsh NE DE 0
M.Brown NE DT 0
K.Van Noy NE OLB 0
M.Butler NE CB 0
J.Richards NE SS 0
S.Gilmore NE CB 0
A.Butler NE DEF 0
P.Chung NE SS 0
D.Wise NE DEF 0
E.Rowe NE CB 0
A.Smith KC QB 31.02
K.Hunt KC RB 45.6
C.West KC RB 9.5
D.Thomas KC WR 1.2
T.Hill KC WR 26.8
T.Kelce KC TE 9.4
A.Wilson KC WR 8.7
D.Harris KC TE 9.5
C.Conley KC WR 6.3
C.Santos KC K 6
D.Colquitt KC P 0
R.Miller KC DT 0
T.Mitchell KC  0
P.Gaines KC CB 0
E.Murray KC CB 0
A.Bailey KC DE 0
D.Johnson KC ILB 0
M.Peters KC CB 0
F.Zombo KC OLB 0
R.Parker KC SS 0
E.Berry KC FS 0
D.Ford KC OLB 0
J.Houston KC OLB 0
B.Logan KC NT 0
D.Sorensen KC DB 0
T.Taylor BUF QB 18.76
L.McCoy BUF RB 20.9
M.Tol

J.Ryan GB ILB 0
K.Brice GB DB 0
H.Clinton-Dix GB FS 0
M.Daniels GB DE 0
R.Wilson SEA QB 8.32
E.Lacy SEA RB 0.3
C.Carson SEA RB 5.9
D.Baldwin SEA WR 10.0
C.Prosise SEA RB 1.1
N.Vannett SEA TE 2.0
P.Richardson SEA WR 9.9
T.Lockett SEA WR 1.8
J.Graham SEA TE 3.8
B.Walsh SEA K 0
J.Ryan SEA P 0
K.Wright SEA OLB 0
R.Sherman SEA CB 0
N.Jones SEA DEF 0
M.Bennett SEA DE 0
T.Garvin SEA LB 0
K.Chancellor SEA SS 0
M.Smith SEA DE 0
S.Griffin SEA DEF 0
C.Avril SEA DE 0
D.Bass SEA OLB 0
E.Thomas SEA FS 0
B.Wagner SEA MLB 0
F.Clark SEA DE 0
J.Reed SEA DT 0
S.Richardson SEA DE 0
J.Lane SEA CB 0
J.Coleman SEA CB 0
B.Hoyer SF QB 3.22
M.Breida SF RB 1.1
C.Hyde SF RB 13.7
K.Juszczyk SF FB 3.7
P.Garcon SF WR 14.1
M.Goodwin SF WR 5.1
G.Kittle SF WR 7.7
T.Taylor SF WR 1.8
A.Robinson SF WR 1.7
R.Robinson SF CB 0
R.Gould SF  0
B.Pinion SF P 0
E.Mitchell SF DT 0
T.Carradine SF DT 0
R.Armstrong SF OLB 0
D.Johnson SF CB 0
K.Williams SF  0
R.Foster SF DEF 0
A.Armstead SF DE 0
J.Tartt SF SS 0
E.Harold SF OLB 0
D.Jon

In [31]:
p._stats

OrderedDict([(u'defense_ffum', 0),
             (u'defense_tkl', 1),
             (u'defense_int', 0),
             (u'defense_ast', 1),
             (u'defense_sk', 0)])